Accompanying Worksheets to Notes in [Introduction to Computational Physics](https://www.amazon.com/Introduction-Computational-Physics-Differential-Simulations/dp/B0GJD4DNNY).

<style>
.box{padding:12px 14px;border-radius:8px;background:#e6f2ff;color:#0b1f33} @media(prefers-color-scheme:dark){  .box{background:#0b2540;color:#e6f2ff} }
</style>
<div class="box">

# Worksheet 7: Floats and Precision

- Understand float precision in Python;
- Demonstrate catastrophic cancellation with a known expression; 
- Propose a numerically stable solution;
- Use arbitrary-precision floating-point arithmetic.

</div>

In [1]:
import numpy as np
import math
import mpmath as mp  # arbitrary precision float math

Consider the following (in)equalities and run them, discuss your findings and create two similar equations that return `(False, True)`:

```{python}
0.1 + 0.2 == 0.3,   0.2 + 0.2 == 0.4
```

In [2]:
# your code: #


In order to get the expected answer, you can use: 
```{python}
isclose(0.1+0.2, 0.3) 
``` 

## Vocabulary

**Accuracy** refers to how close a measurement or calculation is to the true or expected value. **Precision**, on the other hand, relates to the consistency and *reproducibility* of measurements or calculations. Striking the right balance between accuracy and precision is essential for robust computational physics simulations.

## Precision  

Python's standard `float` type, based on double-precision [IEEE 754 format](https://en.wikipedia.org/wiki/IEEE_754), provides up to 15 decimal digits of precision. However, floating-point numbers are inherently approximations due to the way they are represented in binary. Python's float values are represented as 64-bit double-precision values (called a *double* in C).

-   **NaN**: not a number for missing or undefined place holders, use `float('nan')`.
-   $\pm \infty$: infinity, use `float('inf')` and `float('-inf')` \index{infinity}
-   $\pm 0$: these numbers are equal but sometimes distinct


Float may render 50 digits, but only the first 16 are accurate. A comparison of the real $\pi$ value with the `math.pi` or `numpy.pi` shows that.

In the *IEEE 754 double-precision* binary floating-point format, a 64-bit number is stored using three components: (1) Sign bit: This single bit determines the sign of the number (positive or negative), including when the number is zero. (2) Exponent: The exponent field consists of 11 bits and represents an unsigned integer value from 0 to 2047. The actual exponent value is biased, with an exponent value of 1023 representing the actual zero. Exponents range from -1022 to +1023 because exponents of -1023 (all zeros) and +1024 (all ones) are reserved for special numbers. (3) Significand precision: The *significand* (also known as the mantissa) uses 53 bits (52 explicitly stored). This *precision* allows for approximately 15 to 17 significant decimal digits (approximately $2^{-53} \approx 1.11 \cdot 10^{-16}$). Use `sys.float_info` to print all information about `float`.

## Round-off Error

A rational number is not precisely stored into `float`. The conversion to binary format and the limited bits create a **round-off** error. Even seemingly simple numbers like 0.1 can have a round-off error. This error can accumulate.

## Catastrophic Subtractions

In the following extreme example, we would expect the answer to be 100, but instead we find 0.

```{python}
a = 1e60
b = 100
a+b-a, a-a+b, b-a+a, type(a)
```


<style>
.box{padding:12px 14px;border-radius:8px;background:#e6f2ff;color:#0b1f33} @media(prefers-color-scheme:dark){  .box{background:#0b2540;color:#e6f2ff} }
</style>
<div class="box">


## Task 1: Basic Graph

Create an example with catastrophic subtraction; essentially that $a+b-a \neq b$.  Explore large values of $a$ and small values of $b$. You can use list comprehension to explore more space.

</div>

In [3]:
# define a and b, then subtract them from each other.
# your code: #



You can use `mpmath` for arbitrary precision. Here is an example, then add yours.

In [4]:
a = 1e-30
b = 3e30
print(f"{a-b+b=} and should be {a=}")

mp.mp.dps = 80 # set the decimal places
a2 = mp.mpf(a)
b2 = mp.mpf(b)
print(f"{float(a2-b2+b2)=} is the same as {float(a2)=}")

a-b+b=0.0 and should be a=1e-30
float(a2-b2+b2)=1e-30 is the same as float(a2)=1e-30


In [5]:
# your code: #


<style>
.box{padding:12px 14px;border-radius:8px;background:#e6f2ff;color:#0b1f33} @media(prefers-color-scheme:dark){  .box{background:#0b2540;color:#e6f2ff} }
</style>
<div class="box">


## Task 2: Large Integers

Create an integer number `large_a` through addition or multiplication that has at least 1000 digits. Then print the result `large_a` and discuss the length limit of `float` and `int` data types. 

</div>

In [6]:
# create a large integer with at least 1000 digits of length. 
# you can use `len(str(large_a))` to get the number of digits

large_a = 200
print(f"This large number has {len(str(large_a))} digits.")

# your code: #
...

This large number has 3 digits.


Ellipsis

<style>
.box{padding:12px 14px;border-radius:8px;background:#e6f2ff;color:#0b1f33} @media(prefers-color-scheme:dark){  .box{background:#0b2540;color:#e6f2ff} }
</style>
<div class="box">

## Task 3: Choose Algorithm

Given two numbers $x$ and $y$. Compare the two following algorithms: 

1. $z_1 = (x-y)(x+y)$
2. $z_2 = x^2 - y^2$

Mathematically, we have $z_1 = z_2$, but which algorithm is better for computation? Make a prediction. Then, compute the answers for different values of $x$ and $y$ and compare the answer to the accurate answer $z$.
  
</div>

In [7]:
# your code: #


## Numerical Stability

A classic example to show *catastropic substraction* are the solutions $x_i$ of the quadratic equation $(x + x_1)(x + x_2) = x^2 + bx + c$. Taking the square root magnifies the error; the two solutions \index{stability} \index{quadratic equation} \index{subtraction}

$$
2x_{\pm} = -b \pm \sqrt{b^2 - 4c}
$$
A simple (brute-force) implementation would be:

```{python}
def solveQuad(b,c):
  b0 = sqrt(b**2-4*c)
  return -(b+b0)/2, -(b-b0)/2
```

Take $(x+5)(x-4) = x^2 + x - 20$ and `solveQuad(1,-20)` returns `(5.0, -4.0)` as expected. Now, try solving a more complex case: $(x+10^9)(x-10^{-9})$ = $x^2 + (10^9-10^{-9})x -1$ with `solveQuad(1e9-1e-9, -1)` and you obtain one correct and one incorrect answer. This means that this function is *unstable*; it can return an incorrect answer as demonstrated here: \index{unstable}

```{python}
solveQuad(1e9-1e-9, -1)
```

Two separate approaches can be used to fix this, one is that we could check the answer and simply return nothing, when the answer does not check out:

```{python}
def solveQuadWithCheck(b,c):
  b0 = sqrt(b**2-4*c)
  if not isclose((b+b0) * (b-b0),4*c): return None,None
  return -(b+b0)/2, -(b-b0)/2
```

The other approach is to write a *stable* solution to the problem. This requires a bit more analysis. Essentially, the subtraction $b-b_0$ can be avoided, using the fact that $x_{+}x_{-}=c$, we arrive at a numerically stable solution that uses *addition* for the first solution, then *division* for the second solution; thus avoiding *subtraction*. 

```{python}
def solveQuadStable(b,c):
  b0 = sqrt(b**2-4*c)
  x1 = -(b+b0)/2
  x2 = c/x1
  return x1, x2
solveQuadStable(1e9-1e-9, -1)
```

In [8]:
from sympy import *
x = symbols("x")
f= sin(x)/x
series(f, x, 0)

1 - x**2/6 + x**4/120 + O(x**6)

<style>
.box{padding:12px 14px;border-radius:8px;background:#e6f2ff;color:#0b1f33} @media(prefers-color-scheme:dark){  .box{background:#0b2540;color:#e6f2ff} }
</style>
<div class="box">

## Task 4: Small Values

The function  $f(x) = \sin(x)/x$ has the Taylor expansion  for $x \approx 0$ as follows: $1-x^2/6 + x^4/120 + \ldots$ and contains additions and subtractions. Compare  small `float` $x$ values with the accurate results. 

The accurate results can be generated for example as:

``` python
mp.mp.dps = 40
X_VALS = [0.01, 0.001]
f = lambda x: mp.sin(x)/x
[f(mp.mpf(x)) for x in X_VALS]
```

</div>


In [9]:
# your code: #
 

The End.